## Script to bundle all resources in folder as a batch, transaction, collection, or message

- python version 3.6+
- get data from folder
- bundle
- validate and save

In [ ]:
from json import load, dumps, loads
from yaml import safe_load as yloads, safe_dump as ydumps
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhir.resources.bundle import Bundle as B, BundleEntry as BE, BundleEntryRequest as BER, BundleEntryResponse as BERes, BundleEntrySearch as BES 
from fhir.resources.meta import Meta as M
from fhir.resources.extension import Extension as X
import uuid
from datetime import datetime
from pprint import pprint
from pathlib import Path

#===============bundle variables=================
bundle_id = 'sample-valueset-read-transaction-bundle' #'cdex-provider-load-transaction-bundle'
bundle_type = 'transaction' #'collection'
transaction_method = 'GET' # 'PUT' or 'POST' or "GET"
title = False #True # if True add meta extension title and description to bundle
bundle_description = 'This is an Bundle example for the **Patient Access Brands Bundle Profile**. and represents Example 5 - EHR and EHR Customer Hosted Brands Bundles'
bundle_profile = '' #'http://hl7.org/fhir/smart-app-launch/StructureDefinition/patient-access-brands-bundle'
last_updated = True # if True add last updated to bundle


keep_ids = True # if False strip FHIR ids from all resources
use_uuid = True #False # if False use references
base = '' #'https://ehr.example.org'  #  if using reference instead of UUIDs

# in_folder = '/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/bundler/r4/CDex_PAS_Payer_Load'  # copy/paste absolute path
# in_folder = '/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/unbundle_save/out'  # copy/paste absolute path
in_folder = '/Users/ehaas/Documents/FHIR/US-Core/input/resources'
in_path = Path(in_folder)

in_path

###  write to file

In [ ]:
def write_file(r_dict): # write file
    out_path =  Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/bundler/r4')/f'Bundle-{bundle_id}.yml' #in_path / f'Bundle-{bundle_id}.yml'
    print(out_path)
    out_path.write_text(ydumps(r_dict, sort_keys=False))  # dump bundle to file

### open json file and return as dict

In [ ]:


b = B(
    id = bundle_id,
    type = bundle_type,
    timestamp = datetime.now().astimezone().isoformat(),
)

meta = M()
print(meta)
meta.profile = [bundle_profile] if bundle_profile else None

if title:
    
    meta.extension = []
    meta.extension.append ({ 'url': 'http://hl7.org/fhir/StructureDefinition/instance-name', "valueString": f'Bundle {bundle_id.title()}'})
    meta.extension.append({ 'url': 'http://hl7.org/fhir/StructureDefinition/instance-description', "valueMarkdown": bundle_description})

if last_updated:
    meta.lastUpdated = datetime.now().astimezone().isoformat()
    
b.meta = meta
b.json()


In [ ]:
def open_file(f_name): # get files
    
    try:
        return(loads(f_name.read_text(encoding='utf-8')))
    except ValueError as e:

        return(yloads(f_name.read_text()))

### Get files in current path

In [ ]:
files = [path for path in in_path.iterdir() if "valueset" in path.stem.lower() and path.is_file()] #if 'Bundle' not in path.stem and path.is_file()]
for file in files:
    print(file.name)

### create Bundle 'b'  change the id for unique Bundles!!!

### Add resources to bundle

#### Create a mapping of ids to uuids

- get id from resource
- map to uuid (create a dict of {urn, (old_ref)})
- remove id element
- replace all old refs in bundle with new urns


In [ ]:
b.entry = []
ref_map = {}
for i in sorted(files):
    r = open_file(i)
    print(f"r['id'] = {r['id']}  r['resourceType'] = {r['resourceType']}")
    # remove text,id, meta.extension elements from file 
    try:
        r["meta"].pop('extension', None)
    except KeyError:
        pass
    r.pop('text', None) # remove id elements from files
    if keep_ids:
        old_id = r['id'] # map old id to new urn
    else:
        old_id = r.pop('id',None) # map old id to new urn
    if use_uuid:
        try:
            print(r['url'])
            new_urn = r['url']
            r['id'] = new_urn if keep_ids else None # update id to new urn
        except:
            new_urn = uuid.uuid1().urn # new urn for resource
            r['id'] = new_urn.split(':')[-1] if keep_ids else None # update id to new urn}
        # ref_map[new_urn] = f'{r["resourceType"]}/{old_id}'
        # print( f"id_map = {old_id} --> {new_urn}" , '\n')

        
        # print(f"r['id'] = {r['id']}")
        if transaction_method in ['PUT', 'POST']:
            e = dict( 
                fullUrl = new_urn,
                resource = r,
                )
        else:
            e = dict(
                )
    else: # use resources assume the references the ids are correct
        if transaction_method in ['PUT', 'POST']:
            e = dict( 
                fullUrl = f'{base}/{r["resourceType"]}/{old_id}',
                resource = r,
            )
        else:
            e = dict(
            )
    # print(e)
    if bundle_type in ['transaction', 'batch']:
        if transaction_method in ['PUT', 'POST']:
            url = r['resourceType']
        elif transaction_method == 'GET':
            if r["url"]:
                url = f'{r["resourceType"]}?url={r["url"]}'
            else:
                url = f'{r["resourceType"]}?_id={old_id}'   ###### TODO fix this
        e['request'] = dict(
                    method = transaction_method,
                    url = url,
                    )
             
    # if for a PAS request or response Bundle
    if r['resourceType'] in ['ClaimResponse','Claim']:
        b.entry.insert(0,e)
    else:
        b.entry.append(e)
b_json = b.json()
if use_uuid:

    for new_ref, old_ref in ref_map.items():
        b_json = b_json.replace(old_ref,new_ref)
    # print(b_json)
    # b_obj = B.parse_raw(b_json)

print(b_json)

write_file(loads(b_json))
